In [ ]:
import pandas as pd
import numpy as np
import requests
import json
from matplotlib import pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
census_api = requests.get('https://api.census.gov/data/2017/acs/acs5/profile?get=group(DP03)&for=zip%20code%20tabulation%20area:48127&in=state:26&key=367661616b8cd055c91fe0292b28163c9bfeb7a4')
text = census_api.text
print(text)

[["DP03_0001E","DP03_0001EA","DP03_0001M","DP03_0001MA","DP03_0001PE","DP03_0001PEA","DP03_0001PM","DP03_0001PMA","DP03_0002E","DP03_0002EA","DP03_0002M","DP03_0002MA","DP03_0002PE","DP03_0002PEA","DP03_0002PM","DP03_0002PMA","DP03_0003E","DP03_0003EA","DP03_0003M","DP03_0003MA","DP03_0003PE","DP03_0003PEA","DP03_0003PM","DP03_0003PMA","DP03_0004E","DP03_0004EA","DP03_0004M","DP03_0004MA","DP03_0004PE","DP03_0004PEA","DP03_0004PM","DP03_0004PMA","DP03_0005E","DP03_0005EA","DP03_0005M","DP03_0005MA","DP03_0005PE","DP03_0005PEA","DP03_0005PM","DP03_0005PMA","DP03_0006E","DP03_0006EA","DP03_0006M","DP03_0006MA","DP03_0006PE","DP03_0006PEA","DP03_0006PM","DP03_0006PMA","DP03_0007E","DP03_0007EA","DP03_0007M","DP03_0007MA","DP03_0007PE","DP03_0007PEA","DP03_0007PM","DP03_0007PMA","DP03_0008E","DP03_0008EA","DP03_0008M","DP03_0008MA","DP03_0008PE","DP03_0008PEA","DP03_0008PM","DP03_0008PMA","DP03_0009E","DP03_0009EA","DP03_0009M","DP03_0009MA","DP03_0009PE","DP03_0009PEA","DP03_0009PM","DP03

In [ ]:
#Import raw per person exenditure dataset
ppe_df = pd.read_excel('/work/SIADS697_TeamMADScientists/assets/ppe_mi.xlsx', sheet_name = 'PPE')
ppe_df.head()

/usr/local/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,DateExtracted,SCHOOL YEAR,ST,STATENAME,LEAID,ST LEAID,LEA NAME,SCHID,ST SCHID,SCH NAME,...,Students,Teachers,SY1819 SY_STATUS_TEXT,SY1819 SCH_OR_LEA_TYPE,SY 1819 CHARTER_TEXT,SY 1819 GSLO,SY1819 GSHI,SY1819 LEVEL,SY 1819 TITLEI_STATUS,SY1819 CHARTER_LEA_TXT
0,11nov2020 18:23:52,2018-19,MI,MICHIGAN,2600005,MI-13020,BATTLE CREEK PUBLIC SCHOOLS,260000500477,MI-13020-07572,BATTLE CREEK MATHSCI CENTER,...,0,12.00,Open,Regular School,No,08,12,High,Not a Title I school,NaN
1,11nov2020 18:23:52,2018-19,MI,MICHIGAN,2600005,MI-13020,BATTLE CREEK PUBLIC SCHOOLS,260000500807,MI-13020-08574,WK KELLOGG PREPARATORY HIGH SCHOOL,...,195,8.60,Open,Alternative School,No,08,12,High,Title I schoolwide school,NaN
2,11nov2020 18:23:52,2018-19,MI,MICHIGAN,2600005,MI-13020,BATTLE CREEK PUBLIC SCHOOLS,260000501683,MI-13020-09351,ANN J KELLOGG SCHOOL,...,269,18.00,Open,Regular School,No,03,05,Elementary,Title I schoolwide school,NaN
3,11nov2020 18:23:52,2018-19,MI,MICHIGAN,2600005,MI-13020,BATTLE CREEK PUBLIC SCHOOLS,260000501684,MI-13020-09352,LAMORA PARK SCHOOL,...,222,13.55,Open,Regular School,No,PK,02,Elementary,Title I schoolwide school,NaN
4,11nov2020 18:23:52,2018-19,MI,MICHIGAN,2600005,MI-13020,BATTLE CREEK PUBLIC SCHOOLS,260000503830,MI-13020-00223,BATTLE CREEK CENTRAL HIGH SCHOOL,...,1041,56.87,Open,Regular School,No,09,12,High,Title I schoolwide school,NaN


In [ ]:
#Detroit City Zip Codes
zip_codes = [48127, 48239, 48219, 48223, 48228, 48126, 48120, 48217, 48235,
                48227, 48221, 48238, 48204, 48210, 48209, 48206, 48208, 48216,
                48203, 48202, 48201, 48226, 48234, 48212, 48211, 48207, 48205,
                48213, 48214, 48215, 48224, 48236, 48207]

#Irrelevant Columns to drop from ppe_df
cols_to_drop = ['DateExtracted','ST', 'STATENAME', 'LEAID', 'ST LEAID',
       'LEA NAME', 'SCHID', 'ST SCHID', 'ReportUrl', 'ZipCodePlusFour', 'Phone',
       'SY1819 SY_STATUS_TEXT', 'SY 1819 GSLO', 'SY1819 GSHI']

In [ ]:
#Drop blank columns
ppe_df.dropna(axis = 1, how = 'all', inplace = True)
ppe_df = ppe_df.drop(columns = cols_to_drop)

#Create new feature to show number of students per teacher at each school
ppe_df['studentsperteacher'] = ppe_df['Students']/ppe_df['Teachers']

In [ ]:
#ppe_df filtering and transformations

#Filter to only schools in Detroit City zip codes
ppe_df = ppe_df[ppe_df['ZipCode'].isin(zip_codes)]

#Filter out school levels of non interest (elementary schools, middle schools, special ed, etc.)
ppe_df = ppe_df[ppe_df['SY1819 LEVEL'].isin(['High', 'Other'])]
ppe_df = ppe_df[ppe_df['SY1819 SCH_OR_LEA_TYPE'] != 'Special Education School']
ppe_df.drop('SY1819 SCH_OR_LEA_TYPE', axis = 1, inplace = True)

#Only use schools with more than ten students
ppe_df = ppe_df[ppe_df['Students'] > 10]

#Create new binary variable to describe schools who are heavily government funded or not
ppe_df = pd.get_dummies(ppe_df, columns = ['SY 1819 TITLEI_STATUS']).drop(columns = ['SY 1819 TITLEI_STATUS_Title I schoolwide eligible school-No program', 'SY 1819 TITLEI_STATUS_Title I targeted assistance school'])
ppe_df = ppe_df.rename({'SY 1819 TITLEI_STATUS_Not a Title I school': 'Not Title I Funded', 'SY 1819 TITLEI_STATUS_Title I schoolwide school': 'Title I Funded'}, axis = 1)

#Update School name for joining purposes
wsa_indexes = ppe_df[ppe_df['SCH NAME'] == 'WEST SIDE ACADEMY OF INFORMATION TECHNOLOGY AND CYBER SECURI'].index.values
ppe_df.loc[wsa_indexes, 'SCH NAME'] = 'WEST SIDE ACADEMY OF INFORMATION TECHNOLOGY AND CYBER SECURITY'

In [ ]:
#Look at schools left after filtering
ppe_schools = ppe_df['SCH NAME'].unique()
ppe_schools

array(['CRESTWOOD HIGH SCHOOL', 'CESAR CHAVEZ HIGH SCHOOL',
       'DETROIT COMMUNITY SCHOOLS  HIGH SCHOOL',
       'OAKLAND INTERNATIONAL ACADEMY  HIGH SCHOOL',
       'VOYAGEUR COLLEGE PREP', 'STAR INTERNATIONAL ACADEMY',
       'DETROIT EDISON PUBLIC SCHOOL ACADEMY  HIGH SCHOOL',
       'DAVID ELLIS ACADEMY', 'UNIVERSAL ACADEMY',
       'OLD REDFORD ACADEMY  HIGH', 'HOPE OF DETROIT ACADEMY  ELEMENTARY',
       'HOPE OF DETROIT ACADEMY  MIDDLEHIGH', 'JOY PREPARATORY ACADEMY',
       'ADVANCED TECHNOLOGY ACADEMY', 'RIVERSIDE ACADEMY  WEST CAMPUS',
       'FRONTIER INTERNATIONAL ACADEMY',
       'COVENANT HOUSE ACADEMY DETROIT  SOUTHWEST SITE',
       'COVENANT HOUSE ACADEMY DETROIT  EAST SITE',
       'COVENANT HOUSE ACADEMY DETROIT  CENTRAL SITE',
       'ACE ACADEMY SDA GLENDALE LINCOLN WOODWARD',
       'ACE ACADEMY  JEFFERSON SITE', 'CLARA B FORD ACADEMY SDA',
       'VISTA MEADOWS ACADEMY',
       'UNIVERSITY PREPARATORY SCIENCE AND MATH PSAD HIGH SCHOOL',
       'UNIVERSITY PREP

In [ ]:
#ppe_df.groupby('ZipCode')['TotalPpeTotal'].mean().sort_values(ascending = False)

In [ ]:
#Load in raw testing information for 2018-2019 school year
hs_1819_df = pd.read_csv('/work/SIADS697_TeamMADScientists/assets/mstep_access_1819.csv')
#hs_1718_df = pd.read_csv('/work/SIADS697_TeamMADScientists/assets/mstep_access_1718.csv')

In [ ]:
#drop null columns
hs_1819_df.dropna(axis = 1, how = 'all', inplace = True)

In [ ]:
#drop unwanted and uninformative columns from the testing data
cols_to_drop = ['IsdCode', 'IsdName','DistrictCode','DistrictName','BuildingCode', 'CountyCode', 'CountyName', 'EntityType', 'Locale', 'SchoolLevel', 'Grade']
hs_1819_df = hs_1819_df.drop(columns = cols_to_drop)

In [ ]:
#Filter school names for joining dataset on ppe_df
hs_1819_df['BuildingName'] = hs_1819_df.BuildingName.apply(str.upper)
mlk_indexes = hs_1819_df[hs_1819_df['BuildingName'] == 'MARTIN LUTHER KING JR. SENIOR HIGH SCHOOL'].index.values
hs_1819_df.loc[mlk_indexes, 'BuildingName'] = 'MARTIN LUTHER KING JR SENIOR HIGH SCHOOL'

In [ ]:
def filter_groups(text):
    '''
    Specific function for merging purposes. Converts 'Black, ...' entries to 
    'African American' & 'White,...' entries to 'White'
    '''
    if text.startswith('Black'):
        return 'African American'
    if text.startswith('White'):
        return 'White'
    else: return text

In [ ]:
#Alter 'Black' & 'White' testing groups to specific naming convention for grouping
hs_1819_df['TestingGroup'] = hs_1819_df['TestingGroup'].apply(lambda x: filter_groups(x))

In [ ]:
#Look at schools after filtering
hs_1819_df.BuildingName.unique()

array(['ALL BUILDINGS', 'HENDERSON ACADEMY',
       'A.L. HOLMES ACADEMY OF BLENDED LEARNING', 'CODY HIGH SCHOOL',
       'DAVIS AEROSPACE TECHNICAL HIGH SCHOOL AT GOLIGHTLY',
       'OSBORN HIGH SCHOOL',
       'DETROIT INTERNATIONAL ACADEMY FOR YOUNG WOMEN',
       'ANN ARBOR TRAIL MAGNET SCHOOL',
       'BAGLEY ELEMENTARY SCHOOL OF JOURNALISM AND TECHNOLOGY',
       'GOLIGHTLY EDUCATION CENTER', 'ROBERTO CLEMENTE LEARNING ACADEMY',
       'BENNETT ELEMENTARY SCHOOL', 'BUNCHE PREPARATORY ACADEMY',
       'BURNS ELEMENTARY-MIDDLE SCHOOL', 'BURTON INTERNATIONAL ACADEMY',
       'CARSTENS ELEMENTARY-MIDDLE SCHOOL', 'CARVER STEM ACADEMY',
       'CASS TECHNICAL HIGH SCHOOL', 'CENTRAL HIGH SCHOOL',
       'CHRYSLER ELEMENTARY SCHOOL', 'COOKE STEM ACADEMY',
       'MACKENZIE ELEMENTARY-MIDDLE SCHOOL',
       'THURGOOD MARSHALL ELEMENTARY SCHOOL',
       'MUNGER ELEMENTARY-MIDDLE SCHOOL',
       'EARHART ELEMENTARY-MIDDLE SCHOOL',
       'BENJAMIN CARSON HIGH SCHOOL OF SCIENCE AND MEDICINE'

In [ ]:
#Load in raw college enrollment information for 2018-19 school year
college_en = pd.read_csv('/work/SIADS697_TeamMADScientists/assets/college_enrollment_1819.csv')

#drop unwanted columns
college_en.dropna(axis = 1, how = 'all', inplace = True)
cols_to_drop = ['Gender', 'ISD Code', 'ISD OfficialName', 'DistrictCode', 'DistrictOfficialName' ,'BuildingCode', 'CountyCode', 'CountyName', 'EntityType', 'SchoolLevel', 'Locale', 'IHE Type']
college_en.drop(cols_to_drop, axis = 1, inplace = True)

#Clean up school names for merging
college_en['BuildingOfficialName'] = college_en.BuildingOfficialName.apply(str.upper)
mlk_indexes = college_en[college_en['BuildingOfficialName'] == 'MARTIN LUTHER KING JR. SENIOR HIGH SCHOOL'].index.values
college_en.loc[mlk_indexes, 'BuildingOfficialName'] = 'MARTIN LUTHER KING JR SENIOR HIGH SCHOOL'
college_en.head()

,SchoolYear,BuildingOfficialName,Subgroup,Total Graduates (All Students),Total Enrolled in an IHE within 0-6 months,Total % Enrolled in an IHE within 0-6 months
0,2018-2019,ALL BUILDINGS,All Students,1651,863,52.2714
1,2018-2019,ALL BUILDINGS,Asian,42,36,85.7143
2,2018-2019,ALL BUILDINGS,Native Hawaiian or Other Pacific Islander,<10,<10,14.2857
3,2018-2019,ALL BUILDINGS,Students With Disabilities,312,69,22.1154
4,2018-2019,ALL BUILDINGS,Hispanic/Latino,163,33,20.2454


In [ ]:
#Match schools that belong to both datasets
college_schools = college_en.BuildingOfficialName.unique()

college_filter = []
for school in college_schools:
    if school in list(hs_1819_df.BuildingName.unique()):
        college_filter.append(school)

In [ ]:
subgroup_filter = ['All Students', 'White', 'African American']

In [ ]:
college_en = college_en[college_en['BuildingOfficialName'].isin(college_filter)]
college_en = college_en[college_en['Subgroup'].isin(subgroup_filter)]
hs_1819_df = hs_1819_df[hs_1819_df['TestingGroup'].isin(subgroup_filter)]

In [ ]:
def text_to_number(text):
    '''
    Specific funtion for removing unwanted < & > signs from numeric columns
    '''
    if text in ['<10', '< 10']:
        return 5
    elif text in ['<5', '< 5', '<5%']:
        return 1
    elif text == '>95%':
        return 95
    else:
        try: return float(text)
        except: return text

In [ ]:
#Fix numbers as strings in numeric columns
college_en['Total Graduates (All Students)'] = college_en['Total Graduates (All Students)'].apply(lambda x: text_to_number(x))
college_en['Total Enrolled in an IHE within 0-6 months'] = college_en['Total Enrolled in an IHE within 0-6 months'].apply(lambda x: text_to_number(x))
college_en['Total % Enrolled in an IHE within 0-6 months'] = college_en['Total % Enrolled in an IHE within 0-6 months'].apply(lambda x: text_to_number(x))

In [ ]:
#Group by school & subgroups of interest
college_en = college_en.groupby(['BuildingOfficialName', 'Subgroup']).mean().reset_index()

In [ ]:
#Remove unwanted text from numeric columns in testing data
for col in hs_1819_df.columns:
    hs_1819_df[col] = hs_1819_df[col].apply(lambda x: text_to_number(x))

#Group testing data by school and testing group
hs_1819_df = hs_1819_df.groupby(['BuildingName','TestingGroup']).mean().reset_index()

In [ ]:
#Demographic information by zip code
demo_zip = pd.read_excel('/work/SIADS697_TeamMADScientists/assets/capstone_demographics_2.xlsx',engine='openpyxl')
demo_zip_T = demo_zip.T
demo_zip_T.columns = demo_zip_T.iloc[0]
demo_zip_final = demo_zip_T.iloc[1: , :]
demo_zip_final.reset_index(inplace=True)
demo_zip_final.columns = demo_zip_final.columns.str.replace('index', 'zip_code')
demo_zip_final = demo_zip_final.fillna(0).drop_duplicates()

In [ ]:
cols_of_interest = ['zip_code', 'Basic Variables Income - Family Households Mean Family Household Income',
'Basic Variables Educational Attainment - High School Graduate/GED']

demo_zip_final = demo_zip_final[cols_of_interest]

In [ ]:
### JY HS 1819 DF ###
mergedjy = ppe_df.merge(hs_1819_df, left_on= 'SCH NAME', right_on = 'BuildingName')
all_1819_df_jy = mergedjy[mergedjy['TestingGroup'] == 'All Students'] # & (mergedjy['Grade'] == 'All Grades')]

all_1819_df_jy.replace('< 10', 5, inplace = True)
all_1819_df_jy.replace('< 5', 1, inplace = True)
all_1819_df_jy.replace('<10', 5, inplace = True)
all_1819_df_jy.replace('<5', 1, inplace = True)
all_1819_df_jy.replace('<5%', 1, inplace = True)
all_1819_df_jy.replace('>95%', 95, inplace = True)
all_1819_df_jy.iloc[:, -3:] = all_1819_df_jy.iloc[:, -3:].astype(float)
all_1819_df_jy.iloc[:, -16:9] = all_1819_df_jy.iloc[:, -16:9].astype(float)

# Save to csv
all_1819_df_jy.to_csv('/work/assets/saved_data/all_1819_df.csv')

/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/frame.py:4524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/indexing.py:1754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


FileNotFoundError: [Errno 2] No such file or directory: '/work/assets/saved_data/all_1819_df.csv'

JY df ^^^

In [ ]:
#Merge testing data with college enrollment data
merged = hs_1819_df.merge(college_en, left_on = ['BuildingName', 'TestingGroup'],
                                                right_on = ['BuildingOfficialName','Subgroup'])

#Drop redundant columns
merged.drop(['BuildingOfficialName', 'Subgroup'], axis = 1, inplace = True)

#Merge ppe data with combined testing/college enrollment data
merged = merged.merge(ppe_df, left_on= 'BuildingName', right_on = 'SCH NAME')

#Drop redundant columns
merged.drop(['SCH NAME', 'SCHOOL YEAR', 'SY1819 LEVEL', 'SY 1819 CHARTER_TEXT'], axis = 1, inplace = True)

#Merge demographic data onto combined data
merged = merged.merge(demo_zip_final, left_on = 'ZipCode', right_on = 'zip_code')
merged.drop('zip_code', axis = 1, inplace = True)

#merged['ZipCode'] = merged['ZipCode'].astype(str)

In [ ]:
#Group df to compare african american and white students
race_comparison = merged[merged['TestingGroup'] != 'All Students']
race_comparison

,BuildingName,TestingGroup,NumberAboveAverageGrowth,NumberAverageGrowth,NumberBelowAverageGrowth,PercentAboveAverage,PercentAverageGrowth,PercentBelowAverage,TotalIncluded,MeanSGP,...,StreetAddress,City,ZipCode,Students,Teachers,studentsperteacher,Not Title I Funded,Title I Funded,Basic Variables Income - Family Households Mean Family Household Income,Basic Variables Educational Attainment - High School Graduate/GED
1,BENJAMIN CARSON HIGH SCHOOL OF SCIENCE AND MED...,African American,8.333333,27.666667,25.333333,14.933333,44.366667,40.666667,62.333333,39.366667,...,571 Mack Avenue,Detroit,48201,383,23.0,16.652174,0,1,30842.0,2548.0
3,BENJAMIN CARSON HIGH SCHOOL OF SCIENCE AND MED...,White,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,571 Mack Avenue,Detroit,48201,383,23.0,16.652174,0,1,30842.0,2548.0
4,CASS TECHNICAL HIGH SCHOOL,African American,90.666667,165.000000,138.000000,23.000000,41.933333,35.066667,393.666667,45.533333,...,2501 2ND AVE,DETROIT,48201,2393,109.0,21.954128,0,1,30842.0,2548.0
6,CASS TECHNICAL HIGH SCHOOL,White,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,2501 2ND AVE,DETROIT,48201,2393,109.0,21.954128,0,1,30842.0,2548.0
7,DETROIT SCHOOL OF ARTS,African American,13.666667,34.000000,42.666667,15.133333,37.566667,47.300000,90.333333,37.666667,...,123 SELDEN ST,DETROIT,48201,435,29.0,15.000000,0,1,30842.0,2548.0
9,CENTRAL HIGH SCHOOL,African American,5.000000,18.000000,28.666667,6.600000,36.366667,56.800000,50.000000,31.466667,...,2425 Tuxedo Street,Detroit,48206,306,21.5,14.232558,0,1,41940.0,5832.0
11,CODY HIGH SCHOOL,African American,10.000000,37.750000,54.250000,6.900000,37.000000,56.350000,101.000000,31.100000,...,18445 CATHEDRAL ST,DETROIT,48228,704,54.5,12.917431,0,1,42657.0,9790.0
13,CODY HIGH SCHOOL,White,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,18445 CATHEDRAL ST,DETROIT,48228,704,54.5,12.917431,0,1,42657.0,9790.0
14,COMMUNICATION AND MEDIA ARTS HIGH SCHOOL,African American,27.000000,44.333333,28.333333,27.100000,44.500000,28.400000,99.666667,49.766667,...,14771 MANSFIELD ST,DETROIT,48227,577,24.0,24.041667,0,1,45789.0,9083.0
16,DAVIS AEROSPACE TECHNICAL HIGH SCHOOL AT GOLIG...,African American,5.000000,15.666667,18.666667,16.366667,38.166667,45.500000,41.000000,38.533333,...,900 Dickerson St,Detroit,48215,168,12.0,14.000000,0,1,41213.0,2857.0


In [ ]:
#College enrollment between races
college_enroll = px.histogram(race_comparison, x = 'TestingGroup', y = 'Total % Enrolled in an IHE within 0-6 months', histfunc = 'avg')
college_enroll

In [ ]:
#Group schools by zip code for visualization
schools_by_zip = merged.groupby('ZipCode').mean().reset_index()
schools_by_zip

,ZipCode,NumberAboveAverageGrowth,NumberAverageGrowth,NumberBelowAverageGrowth,PercentAboveAverage,PercentAverageGrowth,PercentBelowAverage,TotalIncluded,MeanSGP,Total Graduates (All Students),...,TotalPpeSchoolShare,TotalPpeDistrictShare,TotalPpeTotal,Students,Teachers,studentsperteacher,Not Title I Funded,Title I Funded,Basic Variables Income - Family Households Mean Family Household Income,Basic Variables Educational Attainment - High School Graduate/GED
0,48201,33.041667,62.333333,57.708333,14.833333,31.654167,32.262500,150.708333,31.875000,119.500000,...,8941.4075,5528.4,14469.8075,1149.750000,56.75,18.227363,0,1,30842.0,2548.0
1,48202,5.411765,7.607843,7.941176,11.119608,27.539216,28.129412,15.862745,25.398039,16.666667,...,13634.3900,5528.4,19162.7900,263.000000,24.50,10.734694,0,1,46793.0,3436.0
2,48205,13.750000,32.875000,36.250000,13.100000,31.225000,34.412500,80.375000,30.325000,72.666667,...,13437.3800,5528.4,18965.7800,517.000000,37.00,13.972973,0,1,41187.0,9159.0
3,48206,5.000000,18.000000,29.166667,6.516667,36.000000,57.216667,50.500000,31.200000,30.625000,...,8640.8100,5528.4,14169.2100,306.000000,21.50,14.232558,0,1,41940.0,5832.0
4,48207,41.666667,95.333333,92.000000,18.200000,41.633333,40.200000,229.000000,41.700000,152.000000,...,9375.4300,5528.4,14903.8300,1065.000000,57.00,18.684211,0,1,44864.0,4076.0
5,48208,6.717949,11.538462,11.794872,11.947436,24.522436,23.793590,25.333333,24.267949,49.444444,...,15160.0300,5528.4,20688.4300,350.333333,25.00,13.801613,0,1,30789.0,2147.0
6,48209,21.166667,47.222222,48.694444,13.730556,33.905556,38.205556,116.027778,33.575000,114.444444,...,7240.2800,5528.4,12768.6800,1880.000000,81.00,23.209877,0,1,37205.0,5017.0
7,48214,5.000000,8.666667,15.666667,10.100000,33.800000,56.366667,27.666667,33.666667,23.333333,...,13704.7700,5528.4,19233.1700,153.000000,14.00,10.928571,0,1,49863.0,4541.0
8,48215,5.000000,15.666667,18.666667,16.366667,38.166667,45.500000,41.000000,38.533333,17.500000,...,14891.4600,5528.4,20419.8600,168.000000,12.00,14.000000,0,1,41213.0,2857.0
9,48216,5.000000,12.333333,10.666667,16.666667,43.600000,39.666667,28.333333,41.066667,8.500000,...,11808.1100,5528.4,17336.5100,123.000000,7.00,17.571429,0,1,38777.0,1144.0


In [ ]:
#Visualize percent of students who tested above average by zip code
above_avg = px.bar(schools_by_zip, x = 'ZipCode', y = 'PercentAboveAverage', color = 'TotalPpeTotal', text_auto = True)
above_avg.update_layout(xaxis_type = 'category')
above_avg.update_layout(xaxis={'categoryorder':'total descending'})

48235, 48227 (highest test scores) are neighbors along with 48228 (lowest above average test scores)

Highest & lowest above average test scores are lowest funded schools in Detroit based on PPE.

In [ ]:
test_scores_teachers = px.bar(schools_by_zip, x = 'ZipCode', y = 'studentsperteacher', color = 'MeanSGP', text_auto = True)
test_scores_teachers.update_layout(xaxis_type = 'category')
test_scores_teachers.update_layout(xaxis={'categoryorder':'total descending'})
test_scores_teachers

No zip code with low students per teacher performed well in average test scores. Ex: 48202 is one of the highest funded PPE zip codes in the city, yet they severely underperform on test scores & have a low student/teacher average. This is in center city Detroit.

In [ ]:
avg_growth = px.bar(schools_by_zip, x = 'ZipCode', y = 'PercentAverageGrowth', color = 'TotalPpeTotal',
                    color_continuous_scale = ['Red', 'White', 'Green'], text_auto = True)
avg_growth.update_layout(xaxis_type = 'category')
avg_growth.update_layout(xaxis={'categoryorder':'total descending'})

Center city schools seem to have lowest overall PAG. (48228 outlier w/ connection above)

In [ ]:
import statsmodels.formula.api as smf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# OLS on PAG by zip code

In [ ]:
#pluck variables for linear regression
test_df = schools_by_zip[['PercentAverageGrowth', 'TotalPpeTotal', 'studentsperteacher', 
'Basic Variables Income - Family Households Mean Family Household Income',
'Basic Variables Educational Attainment - High School Graduate/GED']]

test_df

,PercentAverageGrowth,TotalPpeTotal,studentsperteacher,Basic Variables Income - Family Households Mean Family Household Income,Basic Variables Educational Attainment - High School Graduate/GED
0,31.654167,14469.8075,18.227363,30842.0,2548.0
1,27.539216,19162.7900,10.734694,46793.0,3436.0
2,31.225000,18965.7800,13.972973,41187.0,9159.0
3,36.000000,14169.2100,14.232558,41940.0,5832.0
4,41.633333,14903.8300,18.684211,44864.0,4076.0
5,24.522436,20688.4300,13.801613,30789.0,2147.0
6,33.905556,12768.6800,23.209877,37205.0,5017.0
7,33.800000,19233.1700,10.928571,49863.0,4541.0
8,38.166667,20419.8600,14.000000,41213.0,2857.0
9,43.600000,17336.5100,17.571429,38777.0,1144.0


In [ ]:
#Standardize variables for regression
scaler = StandardScaler()
test_scaled = scaler.fit_transform(test_df)

cols_to_rename = {'Basic Variables Income - Family Households Mean Family Household Income': 'mean_family_household_income',
                'Basic Variables Educational Attainment - High School Graduate/GED': 'hs_graduate_ged'}

test_df_scaled = pd.DataFrame(test_scaled, index = test_df.index, columns = test_df.columns)
test_df_scaled.rename(cols_to_rename, axis = 1, inplace = True)
test_df_scaled

,PercentAverageGrowth,TotalPpeTotal,studentsperteacher,mean_family_household_income,hs_graduate_ged
0,-0.412940,-0.576453,0.149206,-1.761756,-1.187089
1,-1.135705,1.179410,-1.569465,0.341216,-0.878884
2,-0.488321,1.105699,-0.826668,-0.397876,1.107445
3,0.350378,-0.688921,-0.767124,-0.298601,-0.047284
4,1.339836,-0.414065,0.253998,0.086898,-0.656754
5,-1.665583,1.750223,-0.865974,-1.768744,-1.326268
6,-0.017498,-1.212924,1.292097,-0.922861,-0.330153
7,-0.036038,1.205742,-1.524994,0.745964,-0.495362
8,0.730939,1.649738,-0.820468,-0.394448,-1.079842
9,1.685269,0.496114,-0.001253,-0.715610,-1.674387


In [ ]:
#Run OLS on variables
percent_growth_reg = smf.ols(formula = 'PercentAverageGrowth ~ TotalPpeTotal+studentsperteacher+mean_family_household_income+hs_graduate_ged', data = test_df_scaled)
result_reg = percent_growth_reg.fit()

ols_robust = result_reg.get_robustcov_results('HC1')
print("OLS RESULTS:",ols_robust.summary())

OLS RESULTS:                              OLS Regression Results                             
Dep. Variable:     PercentAverageGrowth   R-squared:                       0.274
Model:                              OLS   Adj. R-squared:                  0.032
Method:                   Least Squares   F-statistic:                     1.769
Date:                  Sun, 14 Aug 2022   Prob (F-statistic):              0.200
Time:                          16:08:08   Log-Likelihood:                -21.398
No. Observations:                    17   AIC:                             52.80
Df Residuals:                        12   BIC:                             56.96
Df Model:                             4                                         
Covariance Type:                    HC1                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------

No variables statistically significant at p-.05. 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e115903d-6f9d-4ba6-a598-8111e36ee3b2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>